In [ ]:
import plumed
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import math

In [ ]:
### Visualizarion and analysis functions    
def cal_OPES_1DFES(cv,dir,sigma=None,block=10,min_slope_index=None):
    if sigma == None:
        KERNEL = plumed.read_as_pandas(f'../../FES/{dir}/KERNELS')
        sigma = KERNEL.loc[:,f'sigma_{cv}'].iloc[-1]
    !python /root/FES_from_Reweighting.py -f ../../FES/{dir}/OPES-COLVAR -o ../../FES/{dir}/1D_FES -s {sigma}  --temp 310 --cv {cv} --bias opes.bias #--stride 100
    FES = plumed.read_as_pandas(f'../../FES/{dir}/1D_FES') 
    COLVAR = plumed.read_as_pandas(f'../../FES/{dir}/OPES-COLVAR')
    x_start = COLVAR[f'{cv}'].iloc[0]
    if x_start < 0:
        x_start = x_start + 2*math.pi
    x_start = x_start/math.pi*180

    !python /root/FES_from_Reweighting.py -f ../../FES/{dir}/OPES-COLVAR -o ../../FES/{dir}/1D_FES_{block} -s {sigma}  --temp 310 --cv {cv} --bias opes.bias --blocks {block}
    block_number = np.around(len(COLVAR[f'{cv}'])/block)
    error = pd.read_csv(f'../../FES/{dir}/1D_FES_{block}_log',sep='\t',header=None)
        
    block_error = float(error.iloc[:,2])
    print(f'block numbers are {block_number}')
    print(f'block errors are {block_error}')
    ERROR = plumed.read_as_pandas(f'../../FES/{dir}/1D_FES_{block}') 

    plot_FES = []
    plot_CV = []
    plot_uncertainty = []
    shift_FES = []
    shift_CV = []
    shift_uncertainty = []
    for i in range(len(ERROR[f'{cv}'])):
        if ERROR[f'{cv}'][i] < 0:
            ERROR[f'{cv}'][i] = ERROR[f'{cv}'][i]+2*math.pi
            shift_CV.append(ERROR[f'{cv}'][i]/math.pi*180)
            shift_FES.append(ERROR['file.free'][i])
            shift_uncertainty.append(ERROR['uncertainty'][i])
        else:
            plot_CV.append(ERROR[f'{cv}'][i]/math.pi*180)
            plot_FES.append(ERROR['file.free'][i])
            plot_uncertainty.append(ERROR['uncertainty'][i])
    plot_CV = plot_CV+shift_CV
    plot_FES = plot_FES + shift_FES
    plot_FES = np.array(plot_FES)/4.184
    plot_uncertainty = plot_uncertainty + shift_uncertainty
    plot_uncertainty = np.array(plot_uncertainty)/4.184

    !rm ../../FES/{dir}/1D_FES_*_*
    return plot_CV,plot_FES,plot_uncertainty,x_start
def plot_allFES(plot_CV,plot_FES,plot_uncertainty,x_start):
    colors = ['C0','C1','C2']
    plt.plot(figsize=(10,6))
    for i in range(len(plot_CV)):
        plt.plot(plot_CV[i],plot_FES[i],label=['MhOR5-EOL','MhOR5-DEET',r'MhOR5$_{apo}$'][i])
        plt.fill_between(plot_CV[i],np.array(plot_FES[i])-np.array(plot_uncertainty[i]),np.array(plot_FES[i])+np.array(plot_uncertainty[i]),alpha=0.5)
        # plt.vlines(x_start[i],ymin=0,ymax=80,linewidth=1,color=colors[i],alpha=0.8,linestyle='--')
    plt.xlabel(r'$\chi_2^{W158}$ (degree)',fontsize=20)
    plt.ylabel('Free Energy (kcal/mol)',fontsize=20)
    plt.ylim(0,20)
    plt.xlim(0,360)
    plt.xticks(np.arange(0,370,60))
    plt.yticks(np.arange(0,21,5))
    plt.axvspan(np.min(x_start), np.max(x_start), color='gray', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'../../FES/chi21D_FES_all.pdf',dpi=300)
    plt.show()

In [ ]:
Aplot_CV_1,Aplot_FES_1,Aplot_uncertainty_1,Ax_start_1 = cal_OPES_1DFES('torsion_chi1-162',dir='MhOR5_apo',sigma=0.0628,min_slope_index=5,block = 10)
Dplot_CV_1,Dplot_FES_1,Dplot_uncertainty_1,Dx_start_1 = cal_OPES_1DFES('torsion_chi1-162',dir='MhOR5_DEET',sigma=0.0628,min_slope_index=5,block = 10)
Eplot_CV_1,Eplot_FES_1,Eplot_uncertainty_1,E_xstart_1 = cal_OPES_1DFES('torsion_chi1-162',dir='MhOR5_EOL',sigma=0.0628,min_slope_index=5,block = 10)
plot_CV_1 = [Eplot_CV_1,Dplot_CV_1,Aplot_CV_1]
plot_FES_1 = [Eplot_FES_1,Dplot_FES_1,Aplot_FES_1]
plot_uncertainty_1 = [Eplot_uncertainty_1,Dplot_uncertainty_1,Aplot_uncertainty_1]
x_start_1 = [E_xstart_1,Dx_start_1,Ax_start_1]

In [ ]:
Aplot_CV_2,Aplot_FES_2,Aplot_uncertainty_2,Ax_start_2 = cal_OPES_1DFES('torsion_chi2-162',sigma=0.0628,dir='MhOR5_apo',min_slope_index=5,block = 10)
Dplot_CV_2,Dplot_FES_2,Dplot_uncertainty_2,Dx_start_2 = cal_OPES_1DFES('torsion_chi2-162',sigma=0.0628,dir='MhOR5_DEET',min_slope_index=5,block = 10)
Eplot_CV_2,Eplot_FES_2,Eplot_uncertainty_2,E_xstart_2 = cal_OPES_1DFES('torsion_chi2-162',sigma=0.0628,dir='MhOR5_EOL',min_slope_index=5,block = 10)
plot_CV_2 = [Eplot_CV_2,Dplot_CV_2,Aplot_CV_2]
plot_FES_2 = [Eplot_FES_2,Dplot_FES_2,Aplot_FES_2]
plot_uncertainty_2 = [Eplot_uncertainty_2,Dplot_uncertainty_2,Aplot_uncertainty_2]
x_start_2 = [E_xstart_2,Dx_start_2,Ax_start_2]

In [ ]:
plot_allFES(plot_CV_1,plot_FES_1,plot_uncertainty_1,x_start_1)

In [ ]:
plot_allFES(plot_CV_2,plot_FES_2,plot_uncertainty_2,x_start_2)